Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# NASBench-101

This colab accompanies [**NAS-Bench-101: Towards Reproducible Neural Architecture Search**](https://arxiv.org/abs/1902.09635) and the rest of the code at https://github.com/google-research/nasbench.

In this colab, we demonstrate how to use the dataset for simple benchmarking and analysis. The publicly available and free hosted colab instances are sufficient to run this colab.

## Load NASBench library and dataset

In [6]:
# This code was written in TF 1.12 but should be supported all the way through
# TF 1.15. Untested in TF 2.0+.
#%tensorflow_version 1.x

from nasbench import api

# Use nasbench_full.tfrecord for full dataset (run download command above).

nasbench = api.NASBench('nasbench_full.tfrecord')

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 176 seconds


In [1]:
import torch
from torch import nn
#torch.cuda.set_device(1)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")


In [2]:
device

device(type='cuda', index=1)

In [3]:

def Conv3_3(in_channels, out_channels,br=False):
    x=torch.nn.Conv2d(in_channels, out_channels, 3, stride=1, padding=1,  bias=True)
    if br:
        return x
    else:
        return torch.nn.Sequential(x,torch.nn.BatchNorm2d(out_channels),torch.nn.ReLU())

def Conv1_1(in_channels, out_channels,br=False):
    x=torch.nn.Conv2d(in_channels, out_channels, 1, stride=1, padding=0,  bias=True)
    if br:
        return x
    else:
        return torch.nn.Sequential(x,torch.nn.BatchNorm2d(out_channels),torch.nn.ReLU())
def MAXPOOL():
    return nn.MaxPool2d(kernel_size=(3, 3),stride=(1,1),padding=(1,1))

In [4]:
# Standard imports
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

# Useful constants
INPUT = 'input'
OUTPUT = 'output'
CONV3X3 = 'conv3x3-bn-relu'
CONV1X1 = 'conv1x1-bn-relu'
MAXPOOL3X3 = 'maxpool3x3'
CONV5X5= 'conv3x3-bn-relu'
NUM_VERTICES = 7
MAX_EDGES = 9
EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2   # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2   # Input/output vertices are fixed
ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]   # Binary adjacency matrix

## Basic usage

In [39]:
# Query an Inception-like cell from the dataset.
cell = api.ModelSpec(
  matrix=[[0, 1, 0, 1, 0, 0, 0],    # input layer
          [0, 0, 0, 1, 0, 1, 0],    # 1x1 conv
          [0, 0, 0, 0, 0, 0, 0],    # 1x1 conv
          [0, 0, 0, 0, 0, 0, 1],    # 3x3 conv (replaced by two 3x3's)
          [0, 0, 0, 0, 0, 0, 0],    # 3x3 conv (replaced by two 3x3's)
          [0, 0, 0, 0, 0, 0, 0],    # max-pool
          [0, 0, 0, 0, 0, 0, 0]],   # output layer
  # Operations at the vertices of the module, matches order of matrix.
  ops=[INPUT, MAXPOOL3X3 ,CONV1X1, CONV1X1, CONV3X3, CONV3X3, OUTPUT])

# Querying multiple times may yield different results. Each cell is evaluated 3
# times at each epoch budget and querying will sample one randomly.
data = nasbench.query(cell)
print(data["test_accuracy"])

0.8991386294364929


In [50]:
in_channels=128
in_channels/2

64.0

In [5]:
matrix=[  [0, 1, 0, 0, 0, 0, 0],    # input layer
          [0, 0, 1, 0, 0, 0, 0],    # 1x1 conv
          [0, 0, 0, 1, 0, 0, 0],    # 1x1 conv
          [0, 0, 0, 0, 1, 0, 0],    # 3x3 conv (replaced by two 3x3's)
          [0, 1, 0, 0, 0, 1, 0],    # 3x3 conv (replaced by two 3x3's)
          [0, 0, 0, 0, 0, 0, 1],    # max-pool
          [0, 0, 0, 0, 0, 0, 0]]    # output layer
ops=[INPUT, CONV1X1, CONV1X1, CONV3X3, CONV3X3, MAXPOOL3X3, OUTPUT]
pre_list=[[] for i in range(len(matrix))]

# i,j的定义是i->j
# 每个节点都有多/单个输入和一个输出

class Cell(nn.Module):
    def __init__(self,in_channels=128) -> None:
        super(Cell,self).__init__()


        self.reduce_layer=torch.nn.ModuleDict()
        self.normal_layer=torch.nn.ModuleDict()
        self.one_layer=torch.nn.ModuleDict()

        self.reduce_layer["conv1x1-bn-relu"]=Conv1_1(in_channels,int(in_channels/2),True)
        self.reduce_layer["conv3x3-bn-relu"]=Conv3_3(in_channels,int(in_channels/2),True)
        self.reduce_layer['maxpool3x3']=torch.nn.Sequential(MAXPOOL(),Conv1_1(in_channels,int(in_channels/2),True))

        self.normal_layer["conv1x1-bn-relu"]=Conv1_1(int(in_channels/2),int(in_channels/2),True)
        self.normal_layer["conv3x3-bn-relu"]=Conv3_3(int(in_channels/2),int(in_channels/2),True)
        self.normal_layer['maxpool3x3']=MAXPOOL()

        self.proj=Conv1_1(in_channels,in_channels,False)

        for i in range(1,6):
            self.one_layer[str(int(in_channels/2)*i)]=Conv1_1(int((in_channels/2)*i),in_channels)

    def op(self,key,index):
        if index==0:
            return self.reduce_layer[key]
        else:
            return self.normal_layer[key]

    def forward(self,x,matrix,ops):

        output=[ 0 for i in range(len(matrix))]
        output[0]=x
        flag=[False]* len(matrix) # 是否该节点已经输出
        flag[0]=True
        pre_list=[ [] for i in range(len(matrix))]

        for node_index in range(len(matrix)):

            if node_index==len(matrix)-1:# 如果到达最后节点
                combine_list=[]
                #print(pre_list[node_index])
                for index in pre_list[node_index]:
                    if index==0:
                        output[node_index]=self.proj(output[index])
                        flag[node_index]=True
                    else:
                        #print(output[index].shape)
                        combine_list.append(output[index])
                cat_res=torch.cat(combine_list,dim=1)
                #print("___-")
                #print(cat_res.shape)
                if cat_res.shape!=x.shape:
                    cat_res=self.one_layer[str(cat_res.shape[1])](cat_res)
                if flag[node_index]:
                    output[node_index]+=cat_res
                else:
                    output[node_index]=cat_res
                return output[node_index]
            elif node_index!=0:
                for index in pre_list[node_index]:# index 是这个节点前置节点的索引
                    #print(output[index].shape)
                    if flag[index]==True:
                        op_name=ops[node_index]
                        output[node_index]+=self.op(op_name,index)(output[index])
                    else:
                        print(index,"->",node_index,"无前置节点")
                    flag[node_index]=True
            for j in range(node_index+1,len(matrix[node_index])):
                if matrix[node_index][j]!=0:
                    pre_list[j].append(node_index)

def DownSample(in_channels):
    return torch.nn.Sequential(nn.MaxPool2d(kernel_size=(2, 2),stride=(2,2),padding=(0,0)),Conv1_1(in_channels,int(in_channels*2)))

class NasBench101Net(nn.Module):
    def __init__(self) -> None:
        super(NasBench101Net,self).__init__()
        self.stem=torch.nn.Sequential(Conv3_3(3,128,True))
        self.cell1=Cell(128)
        self.cell2=Cell(128)
        self.cell3=Cell(128)
        self.downsample1=DownSample(128)
        self.cell4=Cell(256)
        self.cell5=Cell(256)
        self.cell6=Cell(256)
        self.downsample2=DownSample(256)
        self.cell7=Cell(512)
        self.cell8=Cell(512)
        self.cell9=Cell(512)
        self.gl=torch.nn.AdaptiveAvgPool2d(1)
    def forward(self,x,matrix,op):
        x=self.stem(x)
        #print(x.shape)
        x=self.cell1(x,matrix,op)
        #print(x.shape)
        x=self.cell2(x,matrix,op)
        x=self.cell3(x,matrix,op)
        x=self.downsample1(x)
        x=self.cell4(x,matrix,op)
        x=self.cell5(x,matrix,op)
        x=self.cell6(x,matrix,op)
        x=self.downsample2(x)
        x=self.cell7(x,matrix,op)
        x=self.cell8(x,matrix,op)
        x=self.cell9(x,matrix,op)
        x=self.gl(x)
        return x

In [6]:
model=NasBench101Net()
model=model.to(device)


In [9]:
x=torch.rand(32,3,32,32)
x=x.to(device)
model(x,matrix,ops)

RuntimeError: CUDA out of memory. Tried to allocate 16.00 GiB (GPU 1; 23.94 GiB total capacity; 7.42 GiB already allocated; 15.90 GiB free; 7.43 GiB reserved in total by PyTorch)

In [132]:
cell1=Cell(128).cuda()
cell1(torch.rand(32,128,32,32).cuda(),matrix,ops)

tensor([[[[7.5322e-01, 1.5570e+00, 7.3138e-01,  ..., 4.7972e-01,
           0.0000e+00, 1.1604e+00],
          [9.5154e-02, 7.0585e-01, 1.2186e+00,  ..., 2.0199e+00,
           1.3519e+00, 2.0173e+00],
          [1.2401e+00, 1.8049e+00, 3.6003e-01,  ..., 1.6059e+00,
           1.6122e+00, 1.0763e+00],
          ...,
          [2.6604e-01, 0.0000e+00, 2.2119e+00,  ..., 0.0000e+00,
           0.0000e+00, 9.1857e-02],
          [0.0000e+00, 1.9091e+00, 2.5928e+00,  ..., 1.1249e-01,
           4.3528e-01, 0.0000e+00],
          [4.5834e-01, 8.5469e-01, 2.3238e+00,  ..., 3.7390e-01,
           7.2873e-01, 2.1113e+00]],

         [[2.9929e+00, 3.5621e-01, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 2.3796e+00],
          [1.4303e+00, 3.6735e-01, 1.1648e+00,  ..., 2.4019e-02,
           1.5201e+00, 1.0585e+00],
          [2.6834e+00, 7.7700e-01, 6.1897e-01,  ..., 0.0000e+00,
           3.9466e-01, 9.3798e-01],
          ...,
          [1.1366e+00, 0.0000e+00, 7.8945e-01,  ..., 8.5413

0


TypeError: 'NoneType' object is not callable

In [ ]:
fixed_metrics, computed_metrics = nasbench.get_metrics_from_spec(cell)
print(fixed_metrics)
for epochs in nasbench.valid_epochs:
    for repeat_index in range(len(computed_metrics[epochs])):
        data_point = computed_metrics[epochs][repeat_index]
        print('Epochs trained %d, repeat number: %d' % (epochs, repeat_index + 1))
        print(data_point)


## Example search experiment (random vs. evolution)

In [ ]:
def random_spec():
  """Returns a random valid spec."""
  while True:
    matrix = np.random.choice(ALLOWED_EDGES, size=(NUM_VERTICES, NUM_VERTICES))
    matrix = np.triu(matrix, 1)
    ops = np.random.choice(ALLOWED_OPS, size=(NUM_VERTICES)).tolist()
    ops[0] = INPUT
    ops[-1] = OUTPUT
    spec = api.ModelSpec(matrix=matrix, ops=ops)
    if nasbench.is_valid(spec):
      return spec

def mutate_spec(old_spec, mutation_rate=1.0):
  """Computes a valid mutated spec from the old_spec."""
  while True:
    new_matrix = copy.deepcopy(old_spec.original_matrix)
    new_ops = copy.deepcopy(old_spec.original_ops)

    # In expectation, V edges flipped (note that most end up being pruned).
    edge_mutation_prob = mutation_rate / NUM_VERTICES
    for src in range(0, NUM_VERTICES - 1):
      for dst in range(src + 1, NUM_VERTICES):
        if random.random() < edge_mutation_prob:
          new_matrix[src, dst] = 1 - new_matrix[src, dst]
          
    # In expectation, one op is resampled.
    op_mutation_prob = mutation_rate / OP_SPOTS
    for ind in range(1, NUM_VERTICES - 1):
      if random.random() < op_mutation_prob:
        available = [o for o in nasbench.config['available_ops'] if o != new_ops[ind]]
        new_ops[ind] = random.choice(available)
        
    new_spec = api.ModelSpec(new_matrix, new_ops)
    if nasbench.is_valid(new_spec):
      return new_spec

def random_combination(iterable, sample_size):
  """Random selection from itertools.combinations(iterable, r)."""
  pool = tuple(iterable)
  n = len(pool)
  indices = sorted(random.sample(range(n), sample_size))
  return tuple(pool[i] for i in indices)

def run_random_search(max_time_budget=5e6):
  """Run a single roll-out of random search to a fixed time budget."""
  nasbench.reset_budget_counters()
  times, best_valids, best_tests = [0.0], [0.0], [0.0]
  while True:
    spec = random_spec()
    data = nasbench.query(spec)

    # It's important to select models only based on validation accuracy, test
    # accuracy is used only for comparing different search trajectories.
    if data['validation_accuracy'] > best_valids[-1]:
      best_valids.append(data['validation_accuracy'])
      best_tests.append(data['test_accuracy'])
    else:
      best_valids.append(best_valids[-1])
      best_tests.append(best_tests[-1])

    time_spent, _ = nasbench.get_budget_counters()
    times.append(time_spent)
    if time_spent > max_time_budget:
      # Break the first time we exceed the budget.
      break

  return times, best_valids, best_tests

def run_evolution_search(max_time_budget=5e6,
                         population_size=50,
                         tournament_size=10,
                         mutation_rate=1.0):
  """Run a single roll-out of regularized evolution to a fixed time budget."""
  nasbench.reset_budget_counters()
  times, best_valids, best_tests = [0.0], [0.0], [0.0]
  population = []   # (validation, spec) tuples

  # For the first population_size individuals, seed the population with randomly
  # generated cells.
  for _ in range(population_size):
    spec = random_spec()
    data = nasbench.query(spec)
    time_spent, _ = nasbench.get_budget_counters()
    times.append(time_spent)
    population.append((data['validation_accuracy'], spec))

    if data['validation_accuracy'] > best_valids[-1]:
      best_valids.append(data['validation_accuracy'])
      best_tests.append(data['test_accuracy'])
    else:
      best_valids.append(best_valids[-1])
      best_tests.append(best_tests[-1])

    if time_spent > max_time_budget:
      break

  # After the population is seeded, proceed with evolving the population.
  while True:
    sample = random_combination(population, tournament_size)
    best_spec = sorted(sample, key=lambda i:i[0])[-1][1]
    new_spec = mutate_spec(best_spec, mutation_rate)

    data = nasbench.query(new_spec)
    time_spent, _ = nasbench.get_budget_counters()
    times.append(time_spent)

    # In regularized evolution, we kill the oldest individual in the population.
    population.append((data['validation_accuracy'], new_spec))
    population.pop(0)

    if data['validation_accuracy'] > best_valids[-1]:
      best_valids.append(data['validation_accuracy'])
      best_tests.append(data['test_accuracy'])
    else:
      best_valids.append(best_valids[-1])
      best_tests.append(best_tests[-1])

    if time_spent > max_time_budget:
      break

  return times, best_valids, best_tests
  

In [ ]:
# Run random search and evolution search 10 times each. This should take a few
# minutes to run. Note that each run would have taken days of compute to
# actually train and evaluate if the dataset were not precomputed.
random_data = []
evolution_data = []
for repeat in range(10):
  print('Running repeat %d' % (repeat + 1))
  times, best_valid, best_test = run_random_search()
  random_data.append((times, best_valid, best_test))

  times, best_valid, best_test = run_evolution_search()
  evolution_data.append((times, best_valid, best_test))

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
for times, best_valid, best_test in random_data:
  plt.plot(times, best_valid, label='valid', color='red', alpha=0.5)
  plt.plot(times, best_test, label='test', color='blue', alpha=0.5)

plt.ylabel('accuracy')
plt.xlabel('time spent (seconds)')
plt.ylim(0.92, 0.96)
plt.grid()
plt.title('Random search trajectories (red=validation, blue=test)')


plt.subplot(1, 3, 2)
for times, best_valid, best_test in evolution_data:
  plt.plot(times, best_valid, label='valid', color='red', alpha=0.5)
  plt.plot(times, best_test, label='test', color='blue', alpha=0.5)

plt.ylabel('accuracy')
plt.xlabel('time spent (seconds)')
plt.ylim(0.92, 0.96)
plt.grid()
plt.title('Evolution search trajectories (red=validation, blue=test)')

In [ ]:
# Compare the mean test accuracy along with error bars.
def plot_data(data, color, label, gran=10000, max_budget=5000000):
  """Computes the mean and IQR fixed time steps."""
  xs = range(0, max_budget+1, gran)
  mean = [0.0]
  per25 = [0.0]
  per75 = [0.0]
  
  repeats = len(data)
  pointers = [1 for _ in range(repeats)]
  
  cur = gran
  while cur < max_budget+1:
    all_vals = []
    for repeat in range(repeats):
      while (pointers[repeat] < len(data[repeat][0]) and 
             data[repeat][0][pointers[repeat]] < cur):
        pointers[repeat] += 1
      prev_time = data[repeat][0][pointers[repeat]-1]
      prev_test = data[repeat][2][pointers[repeat]-1]
      next_time = data[repeat][0][pointers[repeat]]
      next_test = data[repeat][2][pointers[repeat]]
      assert prev_time < cur and next_time >= cur

      # Linearly interpolate the test between the two surrounding points
      cur_val = ((cur - prev_time) / (next_time - prev_time)) * (next_test - prev_test) + prev_test
      
      all_vals.append(cur_val)
      
    all_vals = sorted(all_vals)
    mean.append(sum(all_vals) / float(len(all_vals)))
    per25.append(all_vals[int(0.25 * repeats)])
    per75.append(all_vals[int(0.75 * repeats)])
      
    cur += gran
    
  plt.plot(xs, mean, color=color, label=label, linewidth=2)
  plt.fill_between(xs, per25, per75, alpha=0.1, linewidth=0, facecolor=color)

plot_data(random_data, 'red', 'random')
plot_data(evolution_data, 'blue', 'evolution')
plt.legend(loc='lower right')
plt.ylim(0.92, 0.95)
plt.xlabel('total training time spent (seconds)')
plt.ylabel('accuracy')
plt.grid()

## More information

For more information on using the dataset, see the API documentation at https://github.com/google-research/nasbench/blob/master/nasbench/api.py.
